In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import *
from sklearn.metrics import roc_auc_score, log_loss
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
import optuna

In [2]:
train = pd.read_csv('./train.csv').drop(columns=['ID'])
test = pd.read_csv('./test.csv').drop(columns=['ID'])

## 데이터 전처리

In [3]:
def preprocessing(train, test):
    
    # 1. 필요 없는 컬럼 제거
    non_imp_cols = ['난자 혼합 경과일',
    '기증자 정자와 혼합된 난자 수',
    '동결 배아 사용 여부',
    '여성 주 불임 원인',
    '불명확 불임 원인',
    '부부 부 불임 원인',
    '착상 전 유전 진단 사용 여부',
    '시술 유형',
    '남성 부 불임 원인',
    '기증 배아 사용 여부',
    '불임 원인 - 정자 운동성',
    '남성 주 불임 원인',
    '불임 원인 - 자궁내막증',
    '불임 원인 - 정자 농도',
    '여성 부 불임 원인',
    '대리모 여부',
    '불임 원인 - 정자 면역학적 요인',
    '저장된 신선 난자 수',
    '부부 주 불임 원인',
    '착상 전 유전 검사 사용 여부',
    '불임 원인 - 여성 요인',
    '불임 원인 - 자궁경부 문제',
    'PGD 시술 여부',
    'PGS 시술 여부',
    '난자 채취 경과일',
    '난자 해동 경과일',
    '불임 원인 - 정자 형태']
    # '착상 전 유전 검사 사용 여부',
    # '착상 전 유전 진단 사용 여부',
    # '남성 주 불임 원인',
    # '남성 부 불임 원인',
    # '여성 주 불임 원인',
    # '여성 부 불임 원인',
    # '부부 주 불임 원인',
    # '부부 부 불임 원인',
    # '불명확 불임 원인',
    # '불임 원인 - 난관 질환',
    # '불임 원인 - 남성 요인',
    # '불임 원인 - 배란 장애',
    # '불임 원인 - 여성 요인',
    # '불임 원인 - 자궁경부 문제',
    # '불임 원인 - 자궁내막증',
    # '불임 원인 - 정자 농도',
    # '불임 원인 - 정자 면역학적 요인',
    # '불임 원인 - 정자 운동성',
    # '불임 원인 - 정자 형태',
    # 'IVF 시술 횟수',
    # 'DI 시술 횟수',
    # '총 임신 횟수',
    # 'IVF 임신 횟수',
    # 'DI 임신 횟수',
    # '총 출산 횟수',
    # 'IVF 출산 횟수',
    # 'DI 출산 횟수',
    # 'PGD 시술 여부',
    # 'PGS 시술 여부',
    # '대리모 여부',
    # '난자 채취 경과일',
    # '배란 유도 유형',
    # '정자 기증자 나이']
    
    train.drop(columns=non_imp_cols, inplace=True)
    test.drop(columns=non_imp_cols, inplace=True)

    # 미확인 원소 대체
    train['시술 당시 나이'] = train['시술 당시 나이'].replace({'알 수 없음' : None})
    test['시술 당시 나이'] = test['시술 당시 나이'].replace({'알 수 없음' : None})
    train['특정 시술 유형'] = train['특정 시술 유형'].replace({'Unknown' : 'IVF'})
    test['특정 시술 유형'] = test['특정 시술 유형'].replace({'Unknown' : 'IVF'})
    train['난자 출처'] = train['난자 출처'].replace({'알 수 없음' : '본인 제공'})
    test['난자 출처'] = test['난자 출처'].replace({'알 수 없음' : '본인 제공'})

    # 결측치 최빈값 대체
    train = train.apply(lambda x:x.fillna(x.mode()[0]))
    test = test.apply(lambda x:x.fillna(x.mode()[0]))

    categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
    ]
    categorical_columns = [x for x in categorical_columns if x not in non_imp_cols]
    
    for col in categorical_columns:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)


    return train, test, categorical_columns

train, test, categorical_columns = preprocessing(train, test)

In [4]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

In [5]:
# 변환
from sklearn.preprocessing import PowerTransformer

numeric_cols = X.select_dtypes(include=[np.number]).columns

pt = PowerTransformer(method='yeo-johnson', standardize=True)
X[numeric_cols] = pt.fit_transform(X[numeric_cols])
test[numeric_cols] = pt.fit_transform(test[numeric_cols])

# from sklearn.preprocessing import QuantileTransformer

# qt = QuantileTransformer(output_distribution='uniform', random_state=42)
# X[numeric_cols] = qt.fit_transform(X[numeric_cols])
# test[numeric_cols] = qt.fit_transform(test[numeric_cols])

# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X[numeric_cols] = scaler.fit_transform(X[numeric_cols])
# test[numeric_cols] = scaler.fit_transform(test[numeric_cols])

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
X_train_numeric = X_train[numeric_cols]
X_valid_numeric = X_valid[numeric_cols]
test_numeric = test[numeric_cols]

X_train_categoric = X_train[categorical_columns]
X_valid_categoric = X_valid[categorical_columns]
test_categoricric = test[categorical_columns]


## catboost test

In [7]:
def objective(trial):
    # 하이퍼파라미터 제안
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
        'depth': trial.suggest_int('depth', 3, 15),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-2, 10.0),
        'iterations': trial.suggest_int('iterations', 100, 500)
    }
    
    model = CatBoostClassifier(
        **params,
        loss_function='Logloss',
        verbose=0,
        cat_features=categorical_columns,
        random_state=42
    )
    
    model.fit(
        X_train_categoric, y_train,
        eval_set=(X_valid_categoric, y_valid),
        early_stopping_rounds=20
    )
    
    pred_probas = model.predict_proba(X_valid_categoric)[:, 1]
    
    auc = roc_auc_score(y_valid, pred_probas)
    log = log_loss(y_valid, pred_probas)
    return auc, log


In [8]:
study = optuna.create_study(directions=["maximize", "minimize"])
study.optimize(objective, n_trials=20)

print("Pareto front (ROC-AUC) 최적의 트라이얼들:")
for trial in study.best_trials:
    print("Trial values: {} | Params: {}".format(trial.values, trial.params))

[I 2025-02-23 23:57:29,820] A new study created in memory with name: no-name-9119b161-ca38-4bac-9764-364b345ac9af


/tmp/ipykernel_1621/244303575.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
/tmp/ipykernel_1621/244303575.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-2, 10.0),
[I 2025-02-24 00:02:27,119] Trial 0 finished with values: [0.6665871565925245, 0.5273312532071633] and parameters: {'learning_rate': 0.015487140338144011, 'depth': 14, 'l2_leaf_reg': 1.430879617396993, 'iterations': 478}.
/tmp/ipykernel_1621/244303575.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://

KeyboardInterrupt: 

In [ ]:
study.best_trials[0]
# parameters: {'learning_rate': 0.19603157179850195, 'depth': 6, 'l2_leaf_reg': 3.1113757733956238, 'iterations': 402, 'bagging_temperature': 0.056808024089444596, 'random_strength': 3.484431922663257, 'bootstrap_type': 'Bayesian'}. Best is trial 23 with value: 0.4863828367708195.


In [10]:
import pickle

best_params = study.best_trials[0].params

with open('best_params_cat_data_drop2.pkl', 'wb') as f:
    pickle.dump(best_params, f)

print("최적의 파라미터가 best_params.pkl 파일에 저장되었습니다.")

최적의 파라미터가 best_params.pkl 파일에 저장되었습니다.


In [8]:
best_params = study.best_trials[0].params

final_model = CatBoostClassifier(
        **best_params,
        loss_function='Logloss',
        verbose=0,
        cat_features=categorical_columns,
        random_state=42
    )
final_model.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=20)
pred_probas = final_model.predict_proba(X_valid)[:, 1]
print(roc_auc_score(y_valid, pred_probas))




0.7392444579670822


In [ ]:
pred_probas = final_model.predict_proba(test)[:, 1]
# print("예측 확률:", pred_probas)

sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = pred_probas
display(sample_submission)

In [9]:
pred_probas = final_model.predict_proba(test)[:, 1]
print("예측 확률:", pred_probas)
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = pred_probas
sample_submission.to_csv('./submit/submit_39_1.csv', index=False)

예측 확률: [0.00397007 0.00292111 0.15662164 ... 0.40806849 0.19972519 0.00462067]


## lightGBM test

In [4]:
from lightgbm import LGBMClassifier
from sklearn.preprocessing import LabelEncoder

for col in categorical_columns:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.fit_transform(test[col])

X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

from sklearn.preprocessing import PowerTransformer

numeric_cols = X.select_dtypes(include=[np.number]).columns

pt = PowerTransformer(method='yeo-johnson', standardize=True)
X[numeric_cols] = pt.fit_transform(X[numeric_cols])
test[numeric_cols] = pt.fit_transform(test[numeric_cols])

# X_train_numeric = X_train[numeric_cols]
# X_valid_numeric = X_valid[numeric_cols]
# test_numeric = test[numeric_cols]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
def objective(trial):
    # LightGBM 모델의 하이퍼파라미터 제안
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 1.0)
    }
    
    # LGBMClassifier 모델 생성
    model = LGBMClassifier(
        **params,
        random_state=42,
        n_jobs=-1
    )
    
    # 모델 학습 (early stopping 적용)
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='auc'
    )
    
    # 검증 데이터에 대한 예측 확률 (양성 클래스 확률)
    pred_probas = model.predict_proba(X_valid)[:, 1]
    
    # 두 지표 계산
    auc = roc_auc_score(y_valid, pred_probas)
    loss = log_loss(y_valid, pred_probas)
    
    # Optuna 다중 목표: 첫 번째 목표(ROC-AUC)는 최대화, 두 번째 목표(Log Loss)는 최소화
    return auc, loss

In [ ]:
# 3. 다중 목표 최적화를 위한 Optuna Study 생성 및 최적화 수행
study = optuna.create_study(directions=["maximize", "minimize"])
study.optimize(objective, n_trials=20)

# 4. Pareto Front (최적의 트라이얼 목록) 출력
print("Pareto Front (ROC-AUC, Log Loss) 최적의 트라이얼들:")
for trial in study.best_trials:
    print("Trial values (ROC-AUC, Log Loss): {} | Params: {}".format(trial.values, trial.params))

In [6]:
import pickle

# 최적의 파라미터 추출
best_params = study.best_trials[0].params

# best_params를 pickle 파일로 저장
with open('best_params_lightgbm_yj.pkl', 'wb') as f:
    pickle.dump(best_params, f)

print("최적의 파라미터가 best_params.pkl 파일에 저장되었습니다.")

최적의 파라미터가 best_params.pkl 파일에 저장되었습니다.


In [ ]:
# 5. 최적의 파라미터를 사용해 최종 모델 구축 (전체 데이터로 재학습 가능)
best_params = study.best_trials[0].params

final_model = LGBMClassifier(**best_params, random_state=42, n_jobs=-1)
final_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric='auc', callbacks=[early_stopping(stopping_rounds=50)])

# 최종 모델의 검증 예측 및 평가 (예시)
pred_probas_final = final_model.predict_proba(X_valid)[:, 1]
final_auc = roc_auc_score(y_valid, pred_probas_final)
final_loss = log_loss(y_valid, pred_probas_final)
print("\n최종 모델 평가:")
print("ROC-AUC: {:.4f}".format(final_auc))
print("Log Loss: {:.4f}".format(final_loss))

In [ ]:
pred_probas = final_model.predict_proba(test_encoded)[:, 1]
print("예측 확률:", pred_probas)

In [14]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = pred_probas
sample_submission.to_csv('./submit/submit_10.csv', index=False)

## gradient boosting test

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
def objective(trial):
    # GradientBoostingClassifier의 하이퍼파라미터 제안
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'subsample': trial.suggest_uniform('subsample', 0.7, 1.0),
        'max_features': trial.suggest_uniform('max_features', 0.5, 1.0)
    }
    
    # 모델 생성
    model = GradientBoostingClassifier(**params, random_state=42)
    
    # 모델 학습
    model.fit(X_train, y_train)
    
    # 검증 데이터에 대한 예측 확률(양성 클래스)
    pred_probas = model.predict_proba(X_valid)[:, 1]
    
    
    # 두 지표 계산
    auc = roc_auc_score(y_valid, pred_probas)
    loss = log_loss(y_valid, pred_probas)
    
    # Optuna 다중 목표: 첫 번째 목표(ROC-AUC)는 최대화, 두 번째 목표(Log Loss)는 최소화
    return auc, loss

In [8]:
# 3. 다중 목표 최적화를 위한 Optuna Study 생성 및 최적화 수행
study = optuna.create_study(directions=["maximize", "minimize"])
study.optimize(objective, n_trials=30)

# 4. Pareto Front (최적의 트라이얼 목록) 출력
print("Pareto Front (ROC-AUC, Log Loss) 최적의 트라이얼들:")
for trial in study.best_trials:
    print("Trial values (ROC-AUC, Log Loss): {} | Params: {}".format(trial.values, trial.params))

[I 2025-02-21 11:40:06,765] A new study created in memory with name: no-name-c186df39-4b40-4654-80fa-e85b799f1dcf
/tmp/ipykernel_880/2782336391.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.1),
/tmp/ipykernel_880/2782336391.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.7, 1.0),
/tmp/ipykernel_880/2782336391.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'max_features': trial.suggest_uniform('max_features', 0.5, 1

Pareto Front (ROC-AUC, Log Loss) 최적의 트라이얼들:
Trial values (ROC-AUC, Log Loss): [0.7391157019773882, 0.4862781963988753] | Params: {'n_estimators': 511, 'learning_rate': 0.027565392657154242, 'max_depth': 5, 'min_samples_split': 3, 'min_samples_leaf': 19, 'subsample': 0.7088385597011763, 'max_features': 0.5324625168340661}


In [9]:
import pickle

# 최적의 파라미터 추출
best_params = study.best_trials[0].params

# best_params를 pickle 파일로 저장
with open('best_params_gbc.pkl', 'wb') as f:
    pickle.dump(best_params, f)

print("최적의 파라미터가 best_params.pkl 파일에 저장되었습니다.")

최적의 파라미터가 best_params.pkl 파일에 저장되었습니다.


In [ ]:
# 5. 최적의 파라미터를 사용해 최종 모델 구축 (전체 데이터로 재학습 가능)
best_params = study.best_trials[0].params

final_model = GradientBoostingClassifier(**best_params, random_state=42)
    
    # 모델 학습
final_model.fit(X_train, y_train)
# 최종 모델의 검증 예측 및 평가 (예시)
pred_probas_final = final_model.predict_proba(X_valid)[:, 1]
final_auc = roc_auc_score(y_valid, pred_probas_final)
final_loss = log_loss(y_valid, pred_probas_final)
print("\n최종 모델 평가:")
print("ROC-AUC: {:.4f}".format(final_auc))
print("Log Loss: {:.4f}".format(final_loss))

In [ ]:
pred_probas = final_model.predict_proba(test_encoded)[:, 1]
print("예측 확률:", pred_probas)

In [21]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = pred_probas
sample_submission.to_csv('./submit/submit_11.csv', index=False)

## Catboost + lightGBM

In [9]:
def objective(trial):
    params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
            'max_depth': trial.suggest_int('max_depth', 3, 15),
            'num_leaves': trial.suggest_int('num_leaves', 20, 150),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
            'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
            'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        }
        
    # LightGBM Regressor 모델 생성
    model = LGBMRegressor(
        **params,
        random_state=42,
        n_jobs=-1
    )
    # 모델 학습
    model.fit(
        X_train, y_train,
        categorical_feature=categorical_columns
    )
    
    # 검증 데이터 예측
    y_pred = model.predict(X_valid)
    
    # RMSE 계산 (MSE의 제곱근)
    r2 = r2_score(y_valid, y_pred)

    return r2  # RMSE 값을 최소화하는 방향으로 최적화

In [ ]:
# 3. 다중 목표 최적화를 위한 Optuna Study 생성 및 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# 4. 최적의 하이퍼파라미터 및 R² 스코어 출력
print("Best trial:")
trial = study.best_trial
print("  R² Score: {:.4f}".format(trial.value))
print("  Best hyperparameters:")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
best_params = study.best_trials[0].params
best_params

In [ ]:
# 5. 최적의 파라미터를 사용해 최종 모델 구축 (전체 데이터로 재학습 가능)
best_params = study.best_trials[0].params

final_model = LGBMRegressor(**best_params, random_state=42, n_jobs=-1)
final_model.fit(X_train, y_train, categorical_feature=categorical_columns
)

# 최종 모델의 검증 예측 및 평가 (예시)
pred_probas_final = final_model.predict(X_valid)
final_r2 = r2_score(y_valid, pred_probas_final)
print("\n최종 모델 평가:")
print("  R² Score: {:.4f}".format(final_r2))

In [18]:
pred_probas = final_model.predict(test_encoded)
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = pred_probas
sample_submission.to_csv('./submit/submit_16.csv', index=False)